# XGBoost Classification in Python

## Setup

If you are running Databricks Runtime, uncomment the appropriate line in Cmd 3 to install the xgboost library.  
If you are running Databricks Runtime ML, xgboost is already installed. Skip to Cmd 4.

In [0]:
# If you are running Databricks Runtime 7.1 or above, uncomment this line and run this cell:
#%pip install xgboost

# If you are running Databricks Runtime 6.4 to 7.0, uncomment this line and run this cell:
#dbutils.library.installPyPI("xgboost")

## Prepare data

In [0]:
import pandas as pd
import xgboost as xgb

In [0]:
raw_input = pd.read_csv("/dbfs/databricks-datasets/Rdatasets/data-001/csv/datasets/iris.csv",
                        header = 0,
                       names=["item","sepal length","sepal width", "petal length", "petal width","class"])
new_input = raw_input.drop(columns=["item"])
new_input["class"] = new_input["class"].astype('category')
new_input["classIndex"] = new_input["class"].cat.codes
print(new_input)

sepal length sepal width ... class classIndex
0 5.1 3.5 ... setosa 0
1 4.9 3.0 ... setosa 0
2 4.7 3.2 ... setosa 0
3 4.6 3.1 ... setosa 0
4 5.0 3.6 ... setosa 0
5 5.4 3.9 ... setosa 0
6 4.6 3.4 ... setosa 0
7 5.0 3.4 ... setosa 0
8 4.4 2.9 ... setosa 0
9 4.9 3.1 ... setosa 0
10 5.4 3.7 ... setosa 0
11 4.8 3.4 ... setosa 0
12 4.8 3.0 ... setosa 0
13 4.3 3.0 ... setosa 0
14 5.8 4.0 ... setosa 0
15 5.7 4.4 ... setosa 0
16 5.4 3.9 ... setosa 0
17 5.1 3.5 ... setosa 0
18 5.7 3.8 ... setosa 0
19 5.1 3.8 ... setosa 0
20 5.4 3.4 ... setosa 0
21 5.1 3.7 ... setosa 0
22 4.6 3.6 ... setosa 0
23 5.1 3.3 ... setosa 0
24 4.8 3.4 ... setosa 0
25 5.0 3.0 ... setosa 0
26 5.0 3.4 ... setosa 0
27 5.2 3.5 ... setosa 0
28 5.2 3.4 ... setosa 0
29 4.7 3.2 ... setosa 0
.. ... ... ... ... ...
120 6.9 3.2 ... virginica 2
121 5.6 2.8 ... virginica 2
122 7.7 2.8 ... virginica 2
123 6.3 2.7 ... virginica 2
124 6.7 3.3 ... virginica 2
125 7.2 3.2 ... virginica 2
126 6.2 2.8 ... virginica 2
127 6.1 3.0 ... virginica 2
128 6.4 2.8 ... virginica 2
129 7.2 3.0 ... virginica 2
130 7.4 2.8 ... virginica 2
131 7.9 3.8 ... virginica 2
132 6.4 2.8 ... virginica 2
133 6.3 2.8 ... virginica 2
134 6.1 2.6 ... virginica 2
135 7.7 3.0 ... virginica 2
136 6.3 3.4 ... virginica 2
137 6.4 3.1 ... virginica 2
138 6.0 3.0 ... virginica 2
139 6.9 3.1 ... virginica 2
140 6.7 3.1 ... virginica 2
141 6.9 3.1 ... virginica 2
142 5.8 2.7 ... virginica 2
143 6.8 3.2 ... virginica 2
144 6.7 3.3 ... virginica 2
145 6.7 3.0 ... virginica 2
146 6.3 2.5 ... virginica 2
147 6.5 3.0 ... virginica 2
148 6.2 3.4 ... virginica 2
149 5.9 3.0 ... virginica 2

[150 rows x 6 columns]

In [0]:
from sklearn.model_selection import train_test_split
# Split to train/test
training_df, test_df = train_test_split(new_input)

## Train XGBoost Model with Pandas DataFrames

In [0]:
dtrain = xgb.DMatrix(training_df[["sepal length","sepal width", "petal length", "petal width"]], label=training_df["classIndex"])

In [0]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'multi:softmax'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['num_class'] = 6

In [0]:
num_round = 10
bst = xgb.train(param, dtrain, num_round)

## Prediction

In [0]:
dtest = xgb.DMatrix(test_df[["sepal length","sepal width", "petal length", "petal width"]])
ypred = bst.predict(dtest)

In [0]:
from sklearn.metrics import precision_score

pre_score = precision_score(test_df["classIndex"],ypred, average='micro')

print("xgb_pre_score:",pre_score)

('xgb_pre_score:', 0.9736842105263158)